In [1]:
import pandas as pd
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

# ROBOTA
nodes_path = r"C:\Users\relia\Documents\GitHub\Bridges\Data\Nodes_processed.csv"
roads_path = r"C:\Users\relia\Documents\GitHub\Bridges\Data\Roads_processed.csv"
bridges_path = r"C:\Users\relia\Documents\GitHub\Bridges\Data\df_bridges_2024_all.csv"
survey_path = r"C:\Users\relia\Documents\GitHub\Bridges\Data\surveys\Survey.csv"

# Load the CSV data into a DataFrame
df_nodes = pd.read_csv(nodes_path, encoding='utf-8', delimiter=',')
df_roads = pd.read_csv(roads_path, encoding='utf-8', delimiter=',')
df_bridges_all = pd.read_csv(bridges_path, encoding='utf-8', delimiter=',')
df_survey = pd.read_csv(survey_path, encoding='utf-8', delimiter=',')

C:\Users\relia\AppData\Local\Temp\ipykernel_2120\584243297.py:13: DtypeWarning: Columns (5,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_nodes = pd.read_csv(nodes_path, encoding='utf-8', delimiter=',')
C:\Users\relia\AppData\Local\Temp\ipykernel_2120\584243297.py:14: DtypeWarning: Columns (10,11,12,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df_roads = pd.read_csv(roads_path, encoding='utf-8', delimiter=',')


In [2]:
# Assuming df_bridges_all is your DataFrame
columns_to_convert = ['Zemepisná_dĺžka', 'Zemepisná_šírka', 'Výška', 'Normálna_zaťažiteľnosť',
                      'Výhradná_zaťažiteľnosť_t', 'Výnimočná_zaťažiteľnosť_t', 'Počet_otvorov',
                      'Dĺžka_nosnej_konštrukcie_m', 'Dĺžka_premostenia', 'Volná_šírka_mosta',
                      'Šírka_medzi_obrubami_m', 'Plocha_mosnej_konštrukcie_m2', 'Plocha_mosta_m2']

# Convert specified columns to float
df_bridges_all[columns_to_convert] = df_bridges_all[columns_to_convert].apply(
    pd.to_numeric, errors='coerce')

# Convert Rok_postavenia to integer, assign NaN if not possible
df_bridges_all['Rok_postavenia'] = pd.to_numeric(
    df_bridges_all['Rok_postavenia'], errors='coerce')

df_bridges_all = df_bridges_all[df_bridges_all["n_2024"] != 0]

df_bridges_all.columns

Index(['ID_mosta', 'Trieda_PK', 'Číslo_PK', 'Názov_mosta', 'správcovské_číslo',
       'Zemepisná_šírka', 'Zemepisná_dĺžka', 'Výška', 'Druh_konštrukcie',
       'Material', 'Normálna_zaťažiteľnosť', 'Výhradná_zaťažiteľnosť_t',
       'Výnimočná_zaťažiteľnosť_t', 'Spôsob_stanovenia', 'Predmet_premostenia',
       'Rok_postavenia', 'Zaťažovacia_trieda_STN', 'Počet_otvorov',
       'Dĺžka_premostenia', 'Dĺžka_nosnej_konštrukcie_m', 'Volná_šírka_mosta',
       'Šírka_medzi_obrubami_m', 'Plocha_mosnej_konštrukcie_m2',
       'Plocha_mosta_m2', 'Vlastník_úseku', 'Správca_úseku', 'Okres',
       'Počiatočný_uzol', 'Koncový_uzol', 'úsek_ID', 'Kraj', 'n_2012',
       'n_2013', 'n_2014', 'n_2015', 'n_2016', 'n_2017', 'n_2018', 'n_2019',
       'n_2020', 'n_2021', 'n_2022', 'n_2023', 'n_2024'],
      dtype='object')

In [3]:
df_bridges_all["Trieda_PK"].value_counts()

Trieda_PK
cesta III. triedy                      3967
cesta I. triedy                        1913
cesta II. triedy                       1472
diaľnica                                771
miestna neurčená                         56
účelová cesta (UC)                       13
II. trieda - miestna zberná (MZ)          2
III. trieda - miestna obslužná (MO)       1
Name: count, dtype: int64

In [4]:
filtered_bridges_I = df_bridges_all[
    (df_bridges_all["Trieda_PK"] == 'cesta I. triedy')
]

filtered_bridges_II = df_bridges_all[
    (df_bridges_all["Trieda_PK"] == 'cesta II. triedy')
]

filtered_bridges_III = df_bridges_all[
    (df_bridges_all["Trieda_PK"] == 'cesta III. triedy')
]

filtered_bridges_I = filtered_bridges_I[filtered_bridges_I["n_2024"] != 0]
filtered_bridges_II = filtered_bridges_II[filtered_bridges_II["n_2024"] != 0]
filtered_bridges_III = filtered_bridges_III[filtered_bridges_III["n_2024"] != 0]

filtered_bridges_I.shape, filtered_bridges_II.shape, filtered_bridges_III.shape

((1913, 44), (1472, 44), (3967, 44))

In [45]:
import plotly.graph_objs as go
def calculate_relative_condition_counts(data, start_year=2012, end_year=2024):
    relative_counts = pd.DataFrame()

    for year in range(start_year, end_year + 1):
        column_name = f"n_{year}"
        year_data = data[data[column_name] > 0]  # Exclude bridges not assessed
        counts = year_data[column_name].value_counts().rename_axis(
            'Condition').reset_index(name=column_name)
        total = counts[column_name].sum()
        counts[column_name] = (counts[column_name] / total) * 100

        if relative_counts.empty:
            relative_counts = counts
        else:
            relative_counts = pd.merge(
                relative_counts, counts, on='Condition', how='outer')

    relative_counts.fillna(0, inplace=True)
    relative_counts.sort_values('Condition', inplace=True)
    return relative_counts


def calculate_absolute_condition_counts(data, start_year=2012, end_year=2024):
    absolute_counts = pd.DataFrame()

    for year in range(start_year, end_year + 1):
        column_name = f"n_{year}"
        year_data = data[data[column_name] > 0]  # Exclude bridges not assessed
        counts = year_data[column_name].value_counts().rename_axis(
            'Condition').reset_index(name=column_name)

        if absolute_counts.empty:
            absolute_counts = counts
        else:
            absolute_counts = pd.merge(
                absolute_counts, counts, on='Condition', how='outer')

    absolute_counts.fillna(0, inplace=True)
    absolute_counts.sort_values('Condition', inplace=True)
    return absolute_counts


def create_plot(relative_counts, title, filename, start_year=2012, end_year=2024, b=500, h=500):
    color_palette = ['#46a7a2', '#e7a564', '#8bae7c',
                     '#353a29', '#eae1a9', '#c25469', '#2d505d']

    # Set up the years range based on start and end year
    years = [f"n_{year}" for year in range(start_year, end_year + 1)]
    display_years = [str(year) for year in range(start_year, end_year + 1)]

    # Adjust data for plotting
    data_for_plotting = relative_counts.set_index(
        'Condition')[years].T.reset_index()
    data_for_plotting.rename(columns={'index': 'Year'}, inplace=True)
    data_for_plotting['Year'] = display_years

    fig = go.Figure()

    for i, condition in reversed(list(enumerate(relative_counts['Condition'].unique()))):
        fig.add_trace(go.Scatter(
            x=data_for_plotting['Year'],
            y=data_for_plotting[condition],
            mode='lines',
            stackgroup='one',
            name=str(condition),
            line=dict(color=color_palette[i % len(color_palette)])
        ))

    fig.update_layout(
        title=title,
        yaxis_title='percentuálne zastúpenie',
        yaxis=dict(ticksuffix="%", range=[0, 100]),
        font=dict(family="Calibri, sans-serif", size=14),
        legend=dict(
            orientation="h",
            yanchor="top",
            y=1.080,
            xanchor="right",
            x=1
        ),
        xaxis=dict(
            tickvals=[str(year) for year in range(start_year, end_year + 1)],
            ticktext=[
                f"'{str(year)[-2:]}" for year in range(start_year, end_year + 1)]
        ),
        plot_bgcolor='#EEEEEE',
        margin=dict(l=0, r=20, t=75, b=25),
        width=b,
        height=h
    )

    fig.show()

In [14]:
# Calculate relative condition counts for each category
relative_condition_counts_I = calculate_relative_condition_counts(
    filtered_bridges_I)
relative_condition_counts_II = calculate_relative_condition_counts(
    filtered_bridges_II)
relative_condition_counts_III = calculate_relative_condition_counts(
    filtered_bridges_III)

absolute_condition_counts_I = calculate_absolute_condition_counts(
    filtered_bridges_I)

In [15]:
print(absolute_condition_counts_I)

   Condition  n_2012  n_2013  n_2014  n_2015  n_2016  n_2017  n_2018  n_2019  \
0          1     119     122     134     128     142     144     145     137   
1          2     235     238     232     239     233     250     246     244   
2          3     739     725     700     694     645     577     544     491   
3          4     488     485     503     509     547     552     579     594   
4          5     137     136     146     147     162     195     214     258   
5          6      40      51      54      54      66      87      99     116   
6          7       2       3       3       8      14      16       7       8   

   n_2020  n_2021  n_2022  n_2023  n_2024  
0     144     139     131     109      89  
1     248     232     234     179     157  
2     428     386     353     301     289  
3     582     567     573     566     507  
4     304     360     386     464     525  
5     152     187     197     254     316  
6      11      12      15      24      30  


## Absolútne

In [16]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import numpy as np

# Adjust years to match the column names in the format 'n_2012', 'n_2013', etc.
years = [f"n_{year}" for year in range(2012, 2024)]
# Years as integers for model input
X_train = np.array(range(2012, 2024)).reshape(-1, 1)

predictions = {}

# Use PolynomialFeatures to generate polynomial terms
degree = 3  # You can adjust this as needed
poly = PolynomialFeatures(degree=degree)

# Transform X_train to include polynomial terms
X_train_poly = poly.fit_transform(X_train)

for condition in absolute_condition_counts_I['Condition'].unique():
    # Select condition data from 2012 to 2023 for the current condition
    y_train = absolute_condition_counts_I.loc[
        absolute_condition_counts_I['Condition'] == condition, years
    ].values.flatten()

    # Train a Linear Regression model with polynomial features
    model = LinearRegression()
    model.fit(X_train_poly, y_train)

    # Predict for 2024 with polynomial features
    X_test_poly = poly.transform(np.array([[2024]]))
    prediction_2024 = model.predict(X_test_poly)
    predictions[condition] = prediction_2024[0]

# Combine predictions with actual values for comparison
actual_2024 = absolute_condition_counts_I.set_index('Condition')['n_2024']
predictions_df = pd.DataFrame({
    'Condition': predictions.keys(),
    'Predicted_2024': predictions.values(),
    'Actual_2024': actual_2024
}).reset_index(drop=True)

print(predictions_df)

   Condition  Predicted_2024  Actual_2024
0          1       92.814937           89
1          2      147.745463          157
2          3      280.110709          289
3          4      527.468216          507
4          5      522.557239          525
5          6      294.016717          316
6          7       31.006374           30


In [19]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import numpy as np

# Define years for columns and training range
years = [f"n_{year}" for year in range(2012, 2024)]
X_train = np.array(range(2012, 2024)).reshape(-1, 1)

# Prepare a dictionary to store predictions for each condition
predictions = {}

# Polynomial degree setting (adjustable as needed)
degree = 3
poly = PolynomialFeatures(degree=degree)

# Transform X_train to include polynomial terms
X_train_poly = poly.fit_transform(X_train)

# Loop through each condition and perform polynomial regression
for condition in absolute_condition_counts_I['Condition'].unique():
    # Get absolute counts for each condition from 2012 to 2023
    y_train = absolute_condition_counts_I.loc[
        absolute_condition_counts_I['Condition'] == condition, years
    ].values.flatten()

    # Train the Linear Regression model with polynomial features
    model = LinearRegression()
    model.fit(X_train_poly, y_train)

    # Predict for 2024 using the model
    X_test_poly = poly.transform(np.array([[2024]]))
    prediction_2024 = model.predict(X_test_poly)
    predictions[condition] = prediction_2024[0]

# Combine predictions with actual absolute values for 2024
actual_2024 = absolute_condition_counts_I.set_index('Condition')['n_2024']
predictions_df = pd.DataFrame({
    'Condition': predictions.keys(),
    'Predicted_2024': predictions.values(),
    'Actual_2024': actual_2024
}).reset_index(drop=True)

print(predictions_df)

   Condition  Predicted_2024  Actual_2024
0          1       92.814937           89
1          2      147.745463          157
2          3      280.110709          289
3          4      527.468216          507
4          5      522.557239          525
5          6      294.016717          316
6          7       31.006374           30


In [20]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import numpy as np

# Create a copy of the absolute counts DataFrame to avoid modifying it
df_absolute_predictions = absolute_condition_counts_I.copy()

# Extend years to 2035 in the format 'n_2012', 'n_2013', ..., 'n_2035'
years = [f"n_{year}" for year in range(2012, 2036)]
# Prepare X values for model training and predictions (from 2012 to 2035)
X = np.array(range(2012, 2035 + 1)).reshape(-1, 1)

# Initialize PolynomialFeatures with desired degree (adjust as necessary)
degree = 3
poly = PolynomialFeatures(degree=degree)
X_poly = poly.fit_transform(X)

# Initialize dictionary to store predictions for each condition
future_predictions = {f"n_{year}": [] for year in range(2025, 2036)}

for condition in df_absolute_predictions['Condition'].unique():
    # Train model on the entire data from 2012 to 2024 for the current condition
    y = df_absolute_predictions.loc[
        df_absolute_predictions['Condition'] == condition,
        [f"n_{year}" for year in range(2012, 2025)]
    ].values.flatten()

    # Train a Linear Regression model with polynomial features
    model = LinearRegression()
    model.fit(X_poly[:len(y)], y)

    # Predict values from 2025 to 2035
    for year in range(2025, 2036):
        X_test_poly = poly.transform(np.array([[year]]))
        future_predictions[f"n_{year}"].append(model.predict(X_test_poly)[0])

# Append predictions to the DataFrame
for year in range(2025, 2036):
    df_absolute_predictions[f"n_{year}"] = future_predictions[f"n_{year}"]

# Display the updated DataFrame with predictions for 2025-2035
df_absolute_predictions

,Condition,n_2012,n_2013,n_2014,n_2015,n_2016,n_2017,n_2018,n_2019,n_2020,...,n_2026,n_2027,n_2028,n_2029,n_2030,n_2031,n_2032,n_2033,n_2034,n_2035
0,1,119,122,134,128,142,144,145,137,144,...,28.670967,-13.118146,-63.210495,-122.281037,-191.004729,-270.056524,-360.111381,-461.844252,-575.930096,-703.043867
1,2,235,238,232,239,233,250,246,244,248,...,41.684212,-37.047729,-132.749582,-246.986360,-381.323077,-537.324741,-716.556368,-920.582969,-1150.969558,-1409.281146
2,3,739,725,700,694,645,577,544,491,428,...,284.518682,310.350153,357.521607,428.584254,526.089297,652.587943,810.631401,1002.770876,1231.557574,1499.542702
3,4,488,485,503,509,547,552,579,594,582,...,389.510718,300.024253,189.350105,55.585923,-103.170636,-288.821920,-503.270277,-748.418053,-1026.167597,-1338.421255
4,5,137,136,146,147,162,195,214,258,304,...,666.566212,744.439855,826.535178,912.638824,1002.537434,1096.017651,1192.866116,1292.869472,1395.814360,1501.487423
5,6,40,51,54,54,66,87,99,116,152,...,440.500070,520.108290,610.222745,711.535841,824.739984,950.527577,1089.591027,1242.622738,1410.315116,1593.360566
6,7,2,3,3,8,14,16,7,8,11,...,56.072982,74.558079,97.364597,124.941953,157.739570,196.206868,240.793268,291.948190,350.121055,415.761284


In [21]:
import plotly.graph_objs as go

# Years for the x-axis: combine actual years (2012-2024) and predicted years (2025-2035)
years = list(range(2012, 2036))

# Initialize an empty figure
fig = go.Figure()

for condition in df_absolute_predictions['Condition'].unique():
    # Extract actual data from 2012 to 2024
    actual_data = df_absolute_predictions.loc[
        df_absolute_predictions['Condition'] == condition,
        [f"n_{year}" for year in range(2012, 2025)]
    ].values.flatten()

    # Extract predicted data from 2025 to 2035
    predicted_data = df_absolute_predictions.loc[
        df_absolute_predictions['Condition'] == condition,
        [f"n_{year}" for year in range(2025, 2036)]
    ].values.flatten()

    # Combine actual and predicted data for full y-values
    y_values = np.concatenate((actual_data, predicted_data))

    # Create a trace for each condition
    fig.add_trace(go.Scatter(
        x=years, y=y_values,
        mode='lines+markers',
        name=f'Condition {condition}',
        line=dict(shape='spline')
    ))

# Update layout for better readability
fig.update_layout(
    title="Actual and Predicted Absolute Condition Counts (2012-2035)",
    xaxis_title="Year",
    yaxis_title="Absolute Count",
    legend_title="Condition",
)

fig.show()

## Relatívne

In [10]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import numpy as np

# Adjust years to match the column names in the format 'n_2012', 'n_2013', etc.
years = [f"n_{year}" for year in range(2012, 2024)]
# Years as integers for model input
X_train = np.array(range(2012, 2024)).reshape(-1, 1)

predictions = {}

# Use PolynomialFeatures to generate polynomial terms
degree = 3  # You can adjust this to 3 or higher if needed
poly = PolynomialFeatures(degree=degree)

# Transform X_train to include polynomial terms
X_train_poly = poly.fit_transform(X_train)

for condition in relative_condition_counts_I['Condition'].unique():
    # Select condition data from 2012 to 2023 for the current condition
    y_train = relative_condition_counts_I.loc[
        relative_condition_counts_I['Condition'] == condition, years
    ].values.flatten()

    # Train a Linear Regression model with polynomial features
    model = LinearRegression()
    model.fit(X_train_poly, y_train)

    # Predict for 2024 with polynomial features
    X_test_poly = poly.transform(np.array([[2024]]))
    prediction_2024 = model.predict(X_test_poly)
    predictions[condition] = prediction_2024[0]

# Combine predictions with actual values for comparison
actual_2024 = relative_condition_counts_I.set_index('Condition')['n_2024']
predictions_df = pd.DataFrame({
    'Condition': predictions.keys(),
    'Predicted_2024': predictions.values(),
    'Actual_2024': actual_2024
}).reset_index(drop=True)

print(predictions_df)

   Condition  Predicted_2024  Actual_2024
0          1        4.902629     4.652378
1          2        7.792268     8.207005
2          3       15.040182    15.107162
3          4       27.757840    26.502875
4          5       27.430831    27.443806
5          6       15.439813    16.518557
6          7        1.635859     1.568217


In [11]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import numpy as np

# Create a copy of the original DataFrame to avoid modifying it
df_predictions = relative_condition_counts_I.copy()

# Extend years to 2035 in the format 'n_2012', 'n_2013', ..., 'n_2035'
years = [f"n_{year}" for year in range(2012, 2036)]
# Prepare X values for model training and predictions (from 2012 to 2035)
X = np.array(range(2012, 2035 + 1)).reshape(-1, 1)

# Initialize PolynomialFeatures with desired degree (adjust as necessary)
degree = 3
poly = PolynomialFeatures(degree=degree)
X_poly = poly.fit_transform(X)

# Initialize dictionary to store predictions for each condition
future_predictions = {f"n_{year}": [] for year in range(2025, 2036)}

for condition in df_predictions['Condition'].unique():
    # Train model on the entire data from 2012 to 2024 for the current condition
    y = df_predictions.loc[
        df_predictions['Condition'] == condition,
        [f"n_{year}" for year in range(2012, 2025)]
    ].values.flatten()

    # Train a Linear Regression model with polynomial features
    model = LinearRegression()
    model.fit(X_poly[:len(y)], y)

    # Predict values from 2025 to 2035
    for year in range(2025, 2036):
        X_test_poly = poly.transform(np.array([[year]]))
        future_predictions[f"n_{year}"].append(model.predict(X_test_poly)[0])

# Append predictions to the DataFrame
for year in range(2025, 2036):
    df_predictions[f"n_{year}"] = future_predictions[f"n_{year}"]

# Display the updated DataFrame with predictions for 2025-2035
df_predictions

,Condition,n_2012,n_2013,n_2014,n_2015,n_2016,n_2017,n_2018,n_2019,n_2020,...,n_2026,n_2027,n_2028,n_2029,n_2030,n_2031,n_2032,n_2033,n_2034,n_2035
0,1,6.761364,6.931818,7.562077,7.195053,7.849641,7.907743,7.906216,7.413420,7.704655,...,1.538860,-0.590269,-3.120306,-6.082617,-9.508568,-13.429525,-17.876854,-22.881921,-28.476092,-34.690733
1,2,13.352273,13.522727,13.092551,13.434514,12.880044,13.728720,13.413304,13.203463,13.269128,...,2.202978,-1.854579,-6.756054,-12.578527,-19.399079,-27.294788,-36.342737,-46.620003,-58.203668,-71.170812
2,3,41.988636,41.193182,39.503386,39.010680,35.655058,31.685887,29.661941,26.569264,22.899946,...,15.526283,17.412178,20.612414,25.279196,31.564728,39.621216,49.600864,61.655879,75.938466,92.600828
3,4,27.727273,27.556818,28.386005,28.611580,30.237700,30.313015,31.570338,32.142857,31.139647,...,20.354516,15.759274,10.145616,3.424742,-4.492144,-13.693842,-24.269149,-36.306863,-49.895783,-65.124707
4,5,7.784091,7.727273,8.239278,8.263069,8.955224,10.708402,11.668484,13.961039,16.265383,...,34.558201,38.426937,42.483690,46.714578,51.105723,55.643245,60.313265,65.101902,69.995278,74.979513
5,6,2.272727,2.897727,3.047404,3.035413,3.648425,4.777595,5.398037,6.277056,8.132691,...,22.875831,26.926295,31.506711,36.651809,42.396322,48.774980,55.822515,63.573657,72.063139,81.325691
6,7,0.113636,0.170455,0.169300,0.449691,0.773908,0.878638,0.381679,0.432900,0.588550,...,2.944757,3.921769,5.129715,6.592784,8.335163,10.381039,12.754600,15.480034,18.581527,22.083267


In [12]:
import plotly.graph_objs as go

# Years for the x-axis: combine actual years (2012-2024) and predicted years (2025-2035)
years = list(range(2012, 2036))

# Initialize an empty list to hold traces for each condition
fig = go.Figure()

for condition in df_predictions['Condition'].unique():
    # Extract actual data from 2012 to 2024
    actual_data = df_predictions.loc[
        df_predictions['Condition'] == condition,
        [f"n_{year}" for year in range(2012, 2025)]
    ].values.flatten()

    # Extract predicted data from 2025 to 2035
    predicted_data = df_predictions.loc[
        df_predictions['Condition'] == condition,
        [f"n_{year}" for year in range(2025, 2036)]
    ].values.flatten()

    # Combine actual and predicted data for full y-values
    y_values = np.concatenate((actual_data, predicted_data))

    # Create a trace for each condition
    fig.add_trace(go.Scatter(
        x=years, y=y_values,
        mode='lines+markers',
        name=f'Condition {condition}',
        line=dict(shape='spline')
    ))

# Update layout for readability
fig.update_layout(
    title="Actual and Predicted Condition Counts (2012-2035)",
    xaxis_title="Year",
    yaxis_title="Count",
    legend_title="Condition",
)

fig.show()

In [48]:
create_plot(df_predictions, title='Vývoj STS mostov na cestách I. triedy',
            filename='Vyvoj_I', start_year=2012, end_year=2024, b=500, h=500)
create_plot(df_predictions, title='Vývoj STS mostov na cestách I. triedy',
            filename='Vyvoj_I', start_year=2012, end_year=2026, b=575, h=500)

In [39]:
# Calculate the total sum of the values
total_sum = df_predictions["n_2026"].sum()

# Normalize the values to make the sum 100
df_predictions["n_2026"] = (
    df_predictions["n_2026"] / total_sum) * 100

In [41]:
df_predictions["n_2026"]

0     1.538839
1     2.202947
2    15.526061
3    20.354226
4    34.557708
5    22.875504
6     2.944715
Name: n_2026, dtype: float64